<h1 style='font-size:40px'>Spark SQL and DataFrames: Introduction to Built-in Data Sources</h1>

<h2 style='font-size:30px'> Using Spark SQL in Spark Applications</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Neste capítulo, nós ainda lidaremos com outras formas de manipularmos DataFrames no Spark.
        </li>
        <li> 
            Uma funcionalidade que não foi explorada no capítulo anterior é a de se fazer queries SQL nos DF's. Basta criarmos uma View Temporária.
        </li>
    </ul>
</div>

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Chapter 4').getOrCreate()

22/06/10 15:01:56 WARN Utils: Your hostname, veiga-Inspiron resolves to a loopback address: 127.0.1.1; using 192.168.15.21 instead (on interface wlp7s0)
22/06/10 15:01:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/10 15:01:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [46]:
import pandas as pd
import numpy as np
pd.read_csv('https://raw.githubusercontent.com/databricks/LearningSparkV2/master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv',
           dtype={'date':str, 'delay':np.int32, 'distance':np.int32, 'origin':str, 'destination':str}).to_csv('departuredelays.csv')

<h1 style='color:blue'> INICIALIZAR DAQUI!!

In [3]:
# Iremos trabalhar com um dataset sobre vôos de avião.
schema = '`code` STRING, `date` STRING, `delay` INT, `distance` FLOAT, `origin` STRING, `destination` STRING' 
df = spark.read.csv('departuredelays.csv', schema=schema, header=True)

# Montando uma view temporária do DataFrame (é com ela que montaremos as nossas queries).
df.createOrReplaceTempView('us_delay_flights_tbl')

In [4]:
# Quais vôos percorreram mais de 1000 milhas?
spark.sql('''
        SELECT *
        FROM us_delay_flights_tbl
        WHERE distance > 1000
        ORDER BY distance DESC
''').show(5)

22/06/10 15:03:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , date, delay, distance, origin, destination
 Schema: code, date, delay, distance, origin, destination
Expected: code but found: 
CSV file: file:///home/veiga/Documents/Python/PySpark/Learning%20Spark/Chapter%204/departuredelays.csv


+-------+--------+-----+--------+------+-----------+
|   code|    date|delay|distance|origin|destination|
+-------+--------+-----+--------+------+-----------+
|1085920|03061625|   -2|  4330.0|   HNL|        JFK|
|1085580|03021625|   14|  4330.0|   HNL|        JFK|
|1085835|03051625|   -6|  4330.0|   HNL|        JFK|
|1085496|03011625|   -1|  4330.0|   HNL|        JFK|
|1086102|03081530|    4|  4330.0|   HNL|        JFK|
+-------+--------+-----+--------+------+-----------+
only showing top 5 rows



In [5]:
# Vôos entre Sâo Francisco e Chicago com atraso de, no mínimo, 2 horas.
spark.sql('''
    SELECT *
    FROM us_delay_flights_tbl
    WHERE origin = 'SFO'
    AND destination = 'ORD'
    AND delay>=2
    ORDER BY delay DESC
''').show()

22/06/10 15:03:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , date, delay, distance, origin, destination
 Schema: code, date, delay, distance, origin, destination
Expected: code but found: 
CSV file: file:///home/veiga/Documents/Python/PySpark/Learning%20Spark/Chapter%204/departuredelays.csv


+-------+--------+-----+--------+------+-----------+
|   code|    date|delay|distance|origin|destination|
+-------+--------+-----+--------+------+-----------+
| 844767|02190925| 1638|  1604.0|   SFO|        ORD|
| 419836|01031755|  396|  1604.0|   SFO|        ORD|
| 413008|01022330|  326|  1604.0|   SFO|        ORD|
| 424117|01051205|  320|  1604.0|   SFO|        ORD|
| 413489|01190925|  297|  1604.0|   SFO|        ORD|
| 854740|02171115|  296|  1604.0|   SFO|        ORD|
| 420328|01071040|  279|  1604.0|   SFO|        ORD|
| 424118|01051550|  274|  1604.0|   SFO|        ORD|
|1346285|03120730|  266|  1604.0|   SFO|        ORD|
| 422803|01261104|  258|  1604.0|   SFO|        ORD|
| 421509|01161210|  225|  1604.0|   SFO|        ORD|
| 844476|02091800|  223|  1604.0|   SFO|        ORD|
| 422232|01221040|  215|  1604.0|   SFO|        ORD|
|1346286|03121155|  203|  1604.0|   SFO|        ORD|
| 851308|02111256|  197|  1604.0|   SFO|        ORD|
|1335490|03311405|  196|  1604.0|   SFO|      

In [8]:
# Descubra as épocas do ano em que os atrasos costumam ser maiores em vôos entre São Francisco e Chicago.
import pyspark.sql.functions as F
new_df = df.withColumn('date', F.to_date('date', 'MMddHHmm'))
new_df.createOrReplaceTempView('us_delay_flights_tbl')

In [9]:
spark.sql('''
            SELECT MONTH(T.date) MON, AVG(T.delay) AVERAGE_DELAY

            FROM 
            us_delay_flights_tbl T,

            (SELECT AVG(delay) AVERAGE
            FROM us_delay_flights_tbl) V

            WHERE T.origin='SFO'
            AND T.destination='ORD'
            

            GROUP BY MON
''').show()

+---+------------------+
|MON|     AVERAGE_DELAY|
+---+------------------+
|  1|13.622734761120263|
|  2|19.186471663619745|
|  3|13.661927330173775|
+---+------------------+



In [10]:
spark.sql('''
        SELECT TO_DATE(date) a
        FROM us_delay_flights_tbl
        ORDER BY date DESC
            ''').show()

+----------+
|         a|
+----------+
|1970-03-31|
|1970-03-31|
|1970-03-31|
|1970-03-31|
|1970-03-31|
|1970-03-31|
|1970-03-31|
|1970-03-31|
|1970-03-31|
|1970-03-31|
|1970-03-31|
|1970-03-31|
|1970-03-31|
|1970-03-31|
|1970-03-31|
|1970-03-31|
|1970-03-31|
|1970-03-31|
|1970-03-31|
|1970-03-31|
+----------+
only showing top 20 rows



In [11]:
# Aprimorar a query para postar no LinkedIn. Usar um group by para quantificar os casos de 'Delay_Status'.
spark.sql('''
        SELECT code, date, origin, destination,
        CASE
            WHEN delay=0 THEN 'On Time'
            WHEN delay>0 AND delay<60 THEN 'Tolerable Delay'
            WHEN delay>=60  AND delay <120 THEN 'Short Delay'
            WHEN delay>=120 THEN 'Long Delay' 
            ELSE 'Early'
        END Delay_Status
        
        FROM us_delay_flights_tbl
''').show(5)

+----+----------+------+-----------+---------------+
|code|      date|origin|destination|   Delay_Status|
+----+----------+------+-----------+---------------+
|   0|1970-01-01|   ABE|        ATL|Tolerable Delay|
|   1|1970-01-02|   ABE|        DTW|          Early|
|   2|1970-01-02|   ABE|        ATL|          Early|
|   3|1970-01-02|   ABE|        ATL|          Early|
|   4|1970-01-03|   ABE|        ATL|          Early|
+----+----------+------+-----------+---------------+
only showing top 5 rows



22/06/10 15:04:05 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , date, delay, origin, destination
 Schema: code, date, delay, origin, destination
Expected: code but found: 
CSV file: file:///home/veiga/Documents/Python/PySpark/Learning%20Spark/Chapter%204/departuredelays.csv


<h2 style='font-size:30px'> SQL Tables and Views</h2>
<h3 style='font-size:30px;font-style:italic'> Managed Versus Unmanaged Tables</h3>
<div>
    <ul style='font-size:20px'> 
        <li> 
            O Spark admite criarmos dois tipos de tabelas, as gerenciáiveis e as não-gerenciáiveis. Com as primeiras, o Spark administra tanto os seus dados, quanto os metadados. Já com as segundas, apenas os metadados são cuidados; no caso de um DELETE, por exemplo, os dados que povoam a tabela, surpreendentemente, seriam preservados. 
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Creating SQL Databases and Tables</h2>
<div>
    <ul style='font-size:20px'> 
        <li> 
            Todas as tabelas são armazenadas, por padrão, na base de dados <em> default</em>. Por outro lado, é possível gerar uma nova database.
        </li>
    </ul>
</div>

In [12]:
# Criando a DB 'learn_spark_db' e utilizando-a.
spark.sql('CREATE DATABASE learn_spark_db')
spark.sql('USE learn_spark_db')

DataFrame[]

<h3 style='font-size:30px;font-style:italic'> Creating a managed table</h3>

In [13]:
schema = 'code STRING ,date STRING, delay INT, distance FLOAT, origin STRING, destination STRING'
flights_df = spark.read.csv('departuredelays.csv', schema=schema)
flights_df.write.saveAsTable('managed_us_delay_flights_tbl')

AnalysisException: Can not create the managed table('`managed_us_delay_flights_tbl`'). The associated location('file:/home/veiga/Documents/Python/PySpark/Learning%20Spark/Chapter%204/spark-warehouse/learn_spark_db.db/managed_us_delay_flights_tbl') already exists.

In [ ]:
spark.sql('SELECT * FROM managed_us_delay_flights_tbl').show(5)

<h2 style='font-size:30px'> Creating Views</h2>
<div>
    <ul style='font-size:20px'> 
        <li> 
            Como em Bancos de Dados Relacionais, views podem ser criadas. Lembrando, elas não armazenam dados como as tabelas e duram apenas o tempo da sessão.
        </li>
    </ul>
</div>

In [ ]:
# Montando duas views.
spark.sql('''
    CREATE OR REPLACE GLOBAL TEMP VIEW us_origin_airport_SFO_global_tmp_view AS
    SELECT *
    FROM managed_us_delay_flights_tbl
    WHERE origin='SFO'
''')

spark.sql('''
    CREATE OR REPLACE TEMP VIEW us_origin_airport_JFK_global_tmp_view AS
    SELECT *
    FROM managed_us_delay_flights_tbl
    WHERE origin='JFK'
''')

<div>
    <ul style='font-size:20px'> 
        <li> 
            View temporárias globais têm que serem acessadas usando o prefixo global_temp.&lt;nome_view>. Para esse tipo de planilha, o Spark cria uma base de dados temporária.
        </li>
    </ul>
</div>

In [ ]:
spark.read.table('global_temp.us_origin_airport_SFO_global_tmp_view').show(5)

<h2 style='font-size:30px'> Viewing the Metadata</h2>
<div>
    <ul style='font-size:20px'> 
        <li> 
            O atributo "catalog" da SparkSession nos ajuda a visualizar os metadados das bases de dados e tabelas.
        </li>
    </ul>
</div>

In [ ]:
# O 'listColumns' funciona quase como um DESC do Oracle.
spark.catalog.listColumns('managed_us_delay_flights_tbl')

<h2 style='font-size:30px'> Parquet</h2>
<div>
    <ul style='font-size:20px'> 
        <li> 
            Vamos aprender a ler arquivos de extensão "parquet". A vantagem de serem usados no Spark é que eles já possuem o schema da tabela definido em seus metadados.
        </li>
    </ul>
</div>

In [18]:
# Observe que todas as colunas já vêm com o seu devido data type.
spark.read.parquet('train.parquet').printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- country: string (nullable = true)
 |-- url: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- point_of_interest: string (nullable = true)



In [19]:
# Criando uma view temporária a partir de um parquet no Spark SQL.
spark.sql('''
            CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl
            USING parquet 
            OPTIONS (
            path 'train.parquet')
''')

DataFrame[]

<p style='color:red'> Caching SQL Tables